<a href="https://colab.research.google.com/github/JohanMonster/MyRepos/blob/master/IMDB_using_embedding_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import keras
from keras.layers import Dense,Conv1D,Embedding,MaxPooling1D,Dropout,Flatten,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,TensorBoard
from keras.optimizers import Adam
from keras.datasets import imdb
from keras.preprocessing.text import *
import matplotlib.pyplot as plt
from keras import Sequential

In [0]:
device_name = tf.test.gpu_device_name()
print(f"device name{device_name}")

device name/device:GPU:0


In [0]:
from keras.layers import LSTM

In [0]:
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=50000)

In [0]:
word_index=imdb.get_word_index()

In [0]:
max_length_review=(max(len(train_data[i]) for i in range(25000)) )

In [0]:
reverse_index=dict()

In [0]:
reverse_index={index:word for word,index in word_index.items()}

In [0]:
decoded_review=' '.join(reverse_index[i-3] for i in train_data[6])

['?']
['big']
['hair']
['big']
['boobs']
['bad']
['music']
['and']
['a']
['giant']
['safety']
['pin']
['these']
['are']
['the']
['words']
['to']
['best']
['describe']
['this']
['terrible']
['movie']
['i']
['love']
['cheesy']
['horror']
['movies']
['and']
["i've"]
['seen']
['hundreds']
['but']
['this']
['had']
['got']
['to']
['be']
['on']
['of']
['the']
['worst']
['ever']
['made']
['the']
['plot']
['is']
['paper']
['thin']
['and']
['ridiculous']
['the']
['acting']
['is']
['an']
['abomination']
['the']
['script']
['is']
['completely']
['laughable']
['the']
['best']
['is']
['the']
['end']
['showdown']
['with']
['the']
['cop']
['and']
['how']
['he']
['worked']
['out']
['who']
['the']
['killer']
['is']
["it's"]
['just']
['so']
['damn']
['terribly']
['written']
['the']
['clothes']
['are']
['sickening']
['and']
['funny']
['in']
['equal']
['measures']
['the']
['hair']
['is']
['big']
['lots']
['of']
['boobs']
['bounce']
['men']
['wear']
['those']
['cut']
['tee']
['shirts']
['that']
['show']
['o

In [0]:
len(reverse_index)

88584

In [0]:
max_length=1000


In [0]:
vocab_size=20000
input_dim=300

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
X_train=pad_sequences(train_data,maxlen=max_length)
X_test=pad_sequences(test_data,maxlen=max_length)

In [0]:
embedding=Embedding(vocab_size,input_dim,input_length=max_length)

In [0]:
def model():  
# create the model
  model = Sequential()
  model.add(embedding)
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  #model.add(Flatten())
  model.add(LSTM(32,dropout=0.4))
  model.add(Dense(256, activation='relu',kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(Dense(1, activation='sigmoid',kernel_initializer='he_normal'))
  adam=Adam(lr=0.02,beta_1=0.9,beta_2=0.999,decay=1e-3)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  print(model.summary())
  return model
  

In [0]:
def callbacks():
  callbacks=[]
  es=EarlyStopping(monitor='val_loss',min_delta=0,mode='auto',patience=12)
  reduce_lr=ReduceLROnPlateau(monitor='val_loss',patience=12,factor=0.3)
  callbacks=[es,reduce_lr]
  return callbacks

In [0]:
model=model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 750, 300)          6000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 750, 32)           28832     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 375, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total para

In [0]:
model.fit(x=X_train,y=train_labels,epochs=50,batch_size=64,validation_split=0.1,callbacks=callbacks())

Instructions for updating:
Use tf.cast instead.
Train on 22500 samples, validate on 2500 samples
Epoch 1/50
22500/22500 [==============================] - 290s 13ms/step - loss: 0.6444 - acc: 0.6337 - val_loss: 0.6165 - val_acc: 0.6536
Epoch 2/50
22500/22500 [==============================] - 284s 13ms/step - loss: 0.6208 - acc: 0.6651 - val_loss: 0.6281 - val_acc: 0.6592
Epoch 3/50
22500/22500 [==============================] - 286s 13ms/step - loss: 0.6015 - acc: 0.6828 - val_loss: 0.6133 - val_acc: 0.6512
Epoch 4/50
22500/22500 [==============================] - 286s 13ms/step - loss: 0.5901 - acc: 0.6897 - val_loss: 0.6109 - val_acc: 0.6756
Epoch 5/50
22500/22500 [==============================] - 286s 13ms/step - loss: 0.5754 - acc: 0.7080 - val_loss: 0.5916 - val_acc: 0.7012
Epoch 6/50
22500/22500 [==============================] - 286s 13ms/step - loss: 0.5593 - acc: 0.7189 - val_loss: 0.5870 - val_acc: 0.6904
Epoch 7/50
22500/22500 [==============================] - 283s 13ms/s

In [0]:
model.evaluate(X_test,test_labels)

In [0]:
def plot():
  val_loss=history.history['val_loss']
  loss=history.history['loss']
  plt.plot(val_loss)
  plt.plot(loss)
  plt.title('losses')
  plt.show()
  lr=history.history['lr']
  plt.plot(lr)
  plt.title('learning_rates')
  plt.show()
  val_acc=history.history['val_acc']
  acc=history.history['acc']
  plt.plot(val_acc)
  plt.plot(acc)
  plt.title("accuracy curves")
  plt.show()

In [0]:
plot()